In [1]:
%load_ext cython
import matplotlib.pyplot as plt
import meshpy.tet
import numpy as np
%load_ext snakeviz

In [3]:
%%cython -a
import numpy as np
def build_aabb(int nx, int ny, int nz, long [:,:] tetra, double [:,:] nodes, long [:,:] aabb, double [:] step):
    cdef int i, ntetra, j, ix, iy, iz
    cdef double [:,:] bb = np.zeros((2,3))
#     cdef double [:] maxs = np.zeros(3)
    cdef double [:,:] points = np.zeros((4,3))
    ntetra = tetra.shape[0]
    # create a mask for extracting corners from 
    cdef long [:,:] corners = np.array([[0, 1, 0, 0, 1, 0, 1, 1],
                                      [0, 0, 1, 0, 0, 1, 1, 1],
                                      [0, 0, 0, 1, 1, 1, 0, 1]]).astype(int)
    for i in range(ntetra):
        # copy points into tmp array
        for k in range(4):
            for l in range(3):
                points[k,l] = nodes[tetra[i,k],l]
        for l in range(3):
            bb[0,l] = np.min(points[:,l])#,axis=0)
            bb[1,l] = np.max(points[:,l])#,axis=0)
        
        for j in range(8):
            ix = int(bb[corners[0,j],0] // step[0])
            iy = int(bb[corners[1,j],1] // step[1])
            iz = int(bb[corners[2,j],2] // step[2])
            aabb[ix+nx*iy+nx*ny*iz,i] = i
        
    return

In [56]:
%%cython -a
import numpy as np
def find_tetras(double [:,:] nodes, long [:,:] tetras, long [:,:] aabbindx, double [:,:] pos):
    cdef int npts = pos.shape[0]
    cdef int ntetra 
    cdef int i,j,k, l
    cdef double [3] vap, vbp, vab, vac, vad, vbc, vbd
    cdef double va, vb, vc, vd, v, c
    for i in range(npts):
        ntetra = len(aabbindx[i,:])
        for j in range(ntetra):
            
            for k in range(3):
                vap[k] = pos[i,k] - nodes[tetras[aabbindx[i,j],0],k]
                vbp[k] = pos[i,k] - nodes[tetras[aabbindx[i,j],1],k]
                
                vab[k] = nodes[tetras[aabbindx[i,j],1],k] - nodes[tetras[aabbindx[i,j],0],k]
                vac[k] = nodes[tetras[aabbindx[i,j],2],k] - nodes[tetras[aabbindx[i,j],0],k]
                vad[k] = nodes[tetras[aabbindx[i,j],3],k] - nodes[tetras[aabbindx[i,j],0],k]
                vbc[k] = nodes[tetras[aabbindx[i,j],2],k] - nodes[tetras[aabbindx[i,j],1],k]
                vbd[k] = nodes[tetras[aabbindx[i,j],3],k] - nodes[tetras[aabbindx[i,j],1],k]

#                 vab = points[:, :, 1, :] - points[:, :, 0, :]
#                 vac = points[:, :, 2, :] - points[:, :, 0, :]
#                 vad = points[:, :, 3, :] - points[:, :, 0, :]
#                 vbc = points[:, :, 2, :] - points[:, :, 1, :]
#                 vbd = points[:, :, 3, :] - points[:, :, 1, :]
#             vap = pos[i,:] - 
#         vap = pos[i,:] - nodes[] 
#         vap = pos[:,None,:] - points[:, :, 0, :]
#         vbp = pos[:,None,:] - points[:, :, 1, :]
#         # vcp = p - points[:, 2, :]
#         # vdp = p - points[:, 3, :]
#         vab = points[:, :, 1, :] - points[:, :, 0, :]
#         vac = points[:, :, 2, :] - points[:, :, 0, :]
#         vad = points[:, :, 3, :] - points[:, :, 0, :]
#         vbc = points[:, :, 2, :] - points[:, :, 1, :]
#         vbd = points[:, :, 3, :] - points[:, :, 1, :]
        
#         va = np.einsum('ikj, iklj->ikl', vbp, np.cross(vbd, vbc, axisa=2, axisb=2)) / 6.
#         vb = np.einsum('ikj, iklj->ikl', vap, np.cross(vac, vad, axisa=2, axisb=2)) / 6.
#         vc = np.einsum('ikj, iklj->ikl', vap, np.cross(vad, vab, axisa=2, axisb=2)) / 6.
#         vd = np.einsum('ikj, iklj->ikl', vap, np.cross(vab, vac, axisa=2, axisb=2)) / 6.
#         v = np.einsum('ikj, iklj->ikl', vab, np.cross(vac, vad, axisa=2, axisb=2)) / 6.
#         c = np.zeros((va.shape[0],va.shape[1],4))

In [52]:
class TetMesh2:
    def __init__(self,origin,maximum,nodes,tetra,neighbours,nx=2,ny=2,nz=2):
        self.origin = np.array(origin)
        self.maximum = np.array(maximum)
        self.nodes = nodes
        self.tetra = tetra
        self.ntetra = tetra.shape[0]

        self.neighbours = neighbours
        self.step = (self.maximum-self.origin)/(np.array([nx,ny,nz])-1)
        self.aabb = np.zeros((nx*ny*nz,self.ntetra)).astype(int)
        self.aabb[:] = -1
        self.nx = nx
        self.nz = nz
        self.ny = ny
        self._build_aabb()
        self.corners = np.array([[0, 1, 0, 0, 1, 0, 1, 1],
                                      [0, 0, 1, 0, 0, 1, 1, 1],
                                      [0, 0, 0, 1, 1, 1, 0, 1]]).astype(int)
    def _build_aabb(self):
        buffer = self.step*0.1
#         print(self.aabb.shape)
#         tetra = self.tetra
#         nodes = self.nodes
#         ntetra = tetra.shape[0]
#         bb = np.zeros((2,3))
#         step = self.step
#         aabb = self.aabb
#         nx = self.nx
#         ny = self.ny
#         nz = self.nz
#         points = np.zeros((4,3))
#         # create a mask for extracting corners from 
#         corners = np.array([[0, 1, 0, 0, 1, 0, 1, 1],
#                                           [0, 0, 1, 0, 0, 1, 1, 1],
#                                           [0, 0, 0, 1, 1, 1, 0, 1]]).astype(int)
#         for i in range(ntetra):
#             # copy points into tmp array
#             for k in range(4):
#                 for l in range(3):
#                     points[k,l] = nodes[tetra[i,k],l]
#             for l in range(3):
#                 bb[0,l] = np.min(points[:,l])#,axis=0)
#                 bb[1,l] = np.max(points[:,l])#,axis=0)

#             for j in range(8):
#                 ix = int(bb[corners[0,j],0] // step[0])
#                 iy = int(bb[corners[1,j],1] // step[1])
#                 iz = int(bb[corners[2,j],2] // step[2])
#                 aabb[ix+nx*iy+nx*ny*iz,i] = i

#         return
        build_aabb(self.nx,self.ny,self.nz,self.tetra,self.nodes,self.aabb,self.step)
        # now sort
        
    def get_tetra2(self,pos):
        pos = np.array(pos)
        tetras = np.zeros(pos.shape[0]).astype(int)
        bcs = np.zeros((pos.shape[0],4))
        for i in range(pos.shape[0]):
            aabbidx = self.pos_to_aabb_index(pos)
            cells = self.aabb[aabbidx[i],:]
            cells=cells[cells>=0]
            npts = len(cells)
            points = self.nodes[self.tetra[cells]]
            vap = pos[i,:] - points[:, 0, :]
            vbp = pos[i,:] - points[:, 1, :]
            # vcp = p - points[:, 2, :]
            # vdp = p - points[:, 3, :]
            vab = points[:, 1, :] - points[:, 0, :]
            vac = points[:, 2, :] - points[:, 0, :]
            vad = points[:, 3, :] - points[:, 0, :]
            vbc = points[:, 2, :] - points[:, 1, :]
            vbd = points[:, 3, :] - points[:, 1, :]

            va = np.sum(vbp * np.cross(vbd, vbc, axisa=1, axisb=1), axis=1) / 6.
            vb = np.sum(vap * np.cross(vac, vad, axisa=1, axisb=1), axis=1) / 6.
            vc = np.sum(vap * np.cross(vad, vab, axisa=1, axisb=1), axis=1) / 6.
            vd = np.sum(vap * np.cross(vab, vac, axisa=1, axisb=1), axis=1) / 6.
            v = np.sum(vab * np.cross(vac, vad, axisa=1, axisb=1), axis=1) / 6.
            
            c = np.zeros((4, npts))
            c[0, :] = va / v
            c[1, :] = vb / v
            c[2, :] = vc / v
            c[3, :] = vd / v
#             print(c)
            tetras[i] = cells[np.all(c>0,axis=0)]
            #             print(c)
            bcs[i,:] = c[:,np.all(c>0,axis=0)].T
            
        return tetras,bcs
    def get_tetrac(self,pos):
        aabbidx = self.pos_to_aabb_index(pos)
        # get the tetras for each aabb cell where our points are -1 are tetras which 
        # aren't seen in the cell
        cells = self.aabb[aabbidx,:]
        find_tetras(self.nodes, self.tetra, cells, pos)

    def get_tetra(self,pos):
        pos = np.array(pos)
        
        # create containers to store the results (tetra and the bc for each point)
        tetras = np.zeros(pos.shape[0]).astype(int)
        tetras[:] = -1
        bcs = np.zeros((pos.shape[0],4))
        
        # find the aabb cells for each point
        aabbidx = self.pos_to_aabb_index(pos)
        # get the tetras for each aabb cell where our points are -1 are tetras which 
        # aren't seen in the cell
        cells = self.aabb[aabbidx,:]
        
        # create a in array the same shape as cells array
        pos_idx = np.tile(np.arange(cells.shape[0]).T,(cells.shape[1],1)).astype(int).T
    
        # slice the pos and cells arrays for only the tetras that are inside out aabb cells
        # this will create a flat array of points and tetras
        pos2 = pos[pos_idx[cells>0]]
        cells2 = cells[cells>0]
        npts = pos2.shape[0]
        # now calculate the barycentric coordinates for all of these points
        points = self.nodes[self.tetra[cells2]]
        vap = pos2 - points[:, 0, :]
        vbp = pos2 - points[:, 1, :]
        # vcp = p - points[:, 2, :]
        # vdp = p - points[:, 3, :]
        vab = points[:, 1, :] - points[:, 0, :]
        vac = points[:, 2, :] - points[:, 0, :]
        vad = points[:, 3, :] - points[:, 0, :]
        vbc = points[:, 2, :] - points[:, 1, :]
        vbd = points[:, 3, :] - points[:, 1, :]

        va = np.sum(vbp * np.cross(vbd, vbc, axisa=1, axisb=1), axis=1) / 6.
        vb = np.sum(vap * np.cross(vac, vad, axisa=1, axisb=1), axis=1) / 6.
        vc = np.sum(vap * np.cross(vad, vab, axisa=1, axisb=1), axis=1) / 6.
        vd = np.sum(vap * np.cross(vab, vac, axisa=1, axisb=1), axis=1) / 6.
        v = np.sum(vab * np.cross(vac, vad, axisa=1, axisb=1), axis=1) / 6.
        
        c = np.zeros((4, npts))
        c[0, :] = va / v
        c[1, :] = vb / v
        c[2, :] = vc / v
        c[3, :] = vd / v
        mask = np.zeros(cells.shape).astype(bool)
        mask[cells>0] = np.all(c>0,axis=0)
#         mask= np.all(c>0,axis=0)
#         print(mask[0])

#         print('pi',pos_idx[cells>0][np.all(c>0,axis=0)])
        tetras[np.sum(mask,axis=1).astype(bool)] = cells[mask]
#         bc[np.sum(mask,axis=1).astype(bool),:] = c[:,mask].T
        return tetras#,bc

#         return self.aabb[aabbidx,:]
    def pos_to_aabb_index(self,pos):
        ix = pos[:, 0] - self.origin[None, 0]
        iy = pos[:, 1] - self.origin[None, 1]
        iz = pos[:, 2] - self.origin[None, 2]
        ix = ix // self.step[None, 0]
        iy = iy // self.step[None, 1]
        iz = iz // self.step[None, 2]
        return ix.astype(int)+iy.astype(int)*self.nx+iz.astype(int)*self.nx*self.ny
    
    def aabb_index_to_pos(self,global_index):
        global_index = np.array(global_index)
        x_index = global_index % self.nx
        y_index = global_index // self.nx % \
                  self.ny
        z_index = global_index // self.nx // \
                  self.ny
        xcorner = np.array([0, 1, 0, 0, 1, 0, 1, 1])
        ycorner = np.array([0, 0, 1, 0, 0, 1, 1, 1])
        zcorner = np.array([0, 0, 0, 1, 1, 1, 0, 1])
        xcorners = x_index[:, None] + xcorner[None, :]
        ycorners = y_index[:, None] + ycorner[None, :]
        zcorners = z_index[:, None] + zcorner[None, :]
        corner_pos = self.origin[:,None]+self.step[:,None]*np.vstack([xcorners,ycorners,zcorners])
        print(corner_pos)

In [53]:
def build(origin,maximum,n_tetra,nx,ny,nz):
    minx = origin[0]
    miny = origin[1]
    minz = origin[2]
    maxx = maximum[0]
    maxy =  maximum[1]
    maxz =  maximum[2]
    points = np.array([
        (minx, miny, minz),
        (maxx, miny, minz),
        (maxx, maxy, minz),
        (minx, maxy, minz),
        (minx, miny, maxz),
        (maxx, miny, maxz),
        (maxx, maxy, maxz),
        (minx, maxy, maxz)
    ])
#         self.surfaces['top'] = (
#             (minx, miny, maxz), (maxx, miny, maxz), (maxx, maxy, maxz),
#             (minx, maxy, maxz))
#         self.surfaces['bottom'] = (
#             (minx, miny, minz), (maxx, miny, minz), (maxx, maxy, minz),
#             (minx, maxy, minz))

#         self.points = points
#         # calculate the 3 principal components to find the local coordinate
#         # system
#         self.pca.fit(points)
#         # project the points into this coordinate system
#         newp = self.pca.transform(points)

    lengthU = maxx - minx
    lengthV = maxy - miny
    lengthW = maxz - minz
    boxVol = lengthU * lengthW * lengthV
    correction_factor = 1.91
    maxvol = correction_factor * boxVol / n_tetra
    facets = [
        [0, 1, 2, 3],
        [4, 5, 6, 7],
        [0, 4, 5, 1],
        [1, 5, 6, 2],
        [2, 6, 7, 3],
        [3, 7, 4, 0]
    ]

    # create the mesh
    info = meshpy.tet.MeshInfo()
    # use the projected points to build the mesh
    info.set_points(points)
    info.set_facets(facets)
    meshpy_mesh = meshpy.tet.build(info, max_volume=maxvol,
                                   options=meshpy.tet.Options('pqn'))
    nodes = np.array(meshpy_mesh.points)
    elements=  np.array(meshpy_mesh.elements, dtype=np.int64)
    neighbours = np.array(meshpy_mesh.neighbors, dtype=np.int64)
    return TetMesh2(origin,maximum,nodes,elements,neighbours,nx,ny,nz)#    def __init__(self,origin,maximum,nodes,tetra,neighbours,nx=10,ny=10,nz=10):


In [54]:
# for i,t in enumerate([100,1000,5000,7000,10000,15000,20000]):#range(1,10):
#     i=i+2
#     print(i*i*i)
x = np.linspace(0.01,.99,20)
y = np.linspace(0.01,.99,20)
z = np.linspace(0.01,.99,20)
xx,yy,zz = np.meshgrid(x,y,z)
points = np.vstack([xx.flatten(),yy.flatten(),zz.flatten()]).T
mesh2 = build([0,0,0],[1,1,1],3000,5,5,5)
#points = np.mean(mesh.nodes[mesh.tetra],axis=1)
#mask = mesh.get_tetra2(points)#np.random.random((100,3)))
# for t in mesh.tetra:
#     points = mesh.nodes[t]
#     try:
#         mesh.get_tetra([np.mean(points,axis=0)])
#     except:
#         print('exception ',i,np.mean(points,axis=0))
#mesh.get_tetra(np.array([[0.69,0.5,.99]]))#np.random.random((100,3)))
# for i in range(100):
#     print(A[i,A[i,:]>0])

In [57]:
#%%snakeviz
mask2 = mesh2.get_tetrac(points)#np.random.random((100,3)))


In [37]:
#%%snakeviz
mask = mesh2.get_tetra2(points)#np.random.random((100,3)))


In [20]:
print(np.sum(mask2-mask[0]))

-1


In [24]:
points[mask2 == -1]

array([[0.21631579, 0.16473684, 0.21631579]])

In [25]:
np.sum(mask[0] == -1)

0

In [26]:
from LoopStructural.supports.tet_mesh import TetMesh

In [30]:
mesh = TetMesh()
mesh.setup_mesh(np.array([[0,0,0],[1,1,1]]),n_tetra=3000)

In [31]:
mesh.elements_for_array(points)

(array([ 155,  165,  284, ..., 1048,   74,   74]),
 array([ True,  True,  True, ...,  True,  True,  True]))